# Algorithms, Part I
### by Kevin Wayne, Robert Sedgewick

## Week 0: Welcome to Algorithms, Part 1

### Course Introduction
---
* Algorithm: method for solving a problem.
* Data Structure: method to store information.

## Week 1: Union-Find

### Dynamic Connectivity
---
**Steps to Developing a Useable Algorithm**
1. Model the problem.
2. Find an algorithm to solve it.
3. Fast enough? Fits in memory?
4. If not, figure out why.
5. Find a way to address the problem.
6. Iterate until satisfied.

**Dynamic Connectivity Problem**: Given a set of *N* objects, can we support a union command (connect two objects) and a find/connected query (determine whether a path connecting two objects exists) over them?

To model the connections, assume "is connected to" is an equivalence relation satisfying:
* Reflexive: *p* is connected to *p*
* Symmetric: If *p* is connected to *q*, then *q* is connected to *p*.
* Transitive: IF *p* is connected to *q* and *q* is connected to *r*, then *p* is connected to *r*.

A connected component is a maximal set of objects that are mutually connected. From this perspective, the find query checks if two objects are in the same component and the union command replaces components containing two objects with their union.

The initial goal is to design an efficient data structure for union-find given the following considerations:
* Number of objects *N* can be huge.
* Number of operations *M* can be huge.
* Find queries and union commands may be intermixed.

### Quick-Find
---
Also called the "eager approach".

**Data Structure:** Integer array `id[]` of size *N*.

**Interpretation:** Two objects *p* and *q* are connected **iff** their entries in the array are the same.

**Find:** Check if *p* and *q* have the same id.

**Union:** To merge components containing *p* and *q*, change all entries whose id equals `id[p]` to `id[q]`.

**Cost Model:** Number of array accesses (for read or write). In terms of order of growth, initialization is *O*(*N*), union is *O*(*N*), and find is *O*(1).

**Defect:** Union too expensive (*N* array accesses). Trees are flat, but too expensive to keep them flat.

### Quick-Union
---
Also called the "lazy approach".

**Data Structure:** Integer array `id[]` of size *N*.

**Interpretation:** `id[i]` is the parent of `i`. The root of `i` is `id[id[...id[i]...]]`. Checking until the id doesn't change is safe because the algorithm ensures no cycles.

**Find:** Check if *p* and *q* have the same root.

**Union:** To merge components containing *p* and *q*, set the id of *p*'s root to the id of *q*'s root.

**Cost Model:** Number of array accesses (for read or write). In terms of order of growth, initialization is *O*(*N*), union is *O*(*N*), and find is at worst *O*(*N*).

**Defect:** Find too expensive (at worst *N* array accesses). Trees can get tall.

### Quick-Union Improvements
---
**Improvement 1**

Known as "weighted quick-union", the approach is to modify quick-union to avoid tall trees by keeping track of the size of each tree (number of objects), balancing them by linking the root of the smaller tree to the root of the larger tree. Alternatively, can union according to height or "rank".

**Data Structure:** Same as quick-union, but maintain extra array `sz[i]` to count number of objects in the tree rooted at `i`.

**Find:** Identical to quick-union.

**Union:** Modified from quick-union to link root of smaller tree to root of larger tree and update the `sz[]` array.

**Proposition:** Depth of any node *x* is at most lg*N* (base 2).
>Proof: The depth of *x* increases by 1 when tree *T*1 containing *x* is merged into another tree *T*2. The size of the tree containing *x* at least doubles since |*T*2| ≥ |*T*1|, but it can only double at most lg*N* times.

**Running Time:** Find takes time proportional to depth of *p* and *q*. Union takes constant time, given roots. In terms of order of growth, initialization is *O*(*N*), union is *O*(lg*N*) (including the cost of finding the roots), and find is at worst *O*(lg*N*), based on the proposition.

**Improvement 2**

Known as "path compression", modify the algorithm such that after computing the root of *p*, set the id of each examined node to point to that root. This only costs constant time.

**Proposition:** [Hopcroft-Ulman, Tarjan] Starting from an empty data structure, any sequence of *M* union-find ops on *N* objects makes ≤ *c*(*N* + *M*lg\**N*) array accesses, where lg\* is the iterative log function. Analysis can be improved to *N* + *M*α(*M*,*N*).

Is there a linear-time algorithm for *M* union-find ops on *N* objects? No, weighted quick-union with path compression (WQUPC) is not quite linear in theory, but it might as well be in practice.

In summary,

|           Algorithm            | Worst-Case Time  |
| ------------------------------ | :--------------: |
| Quick-Find                     | *M* *N*          |
| Quick-Union (QU)               | *M* *N*          |
| Weighted QU                    | *N* + *M*log*N*  |
| QU + Path Compression          | *N* + *M*log*N*  |
| Weighted QU + Path Compression | *N* + *M*lg\**N* |

Ex. For 10^9 union-find ops on 10^9 objects, WQUPC reduces the time from 30 years to 6 seconds.

### Union-Find Applications
---
**Percolation**

A model for many physical systems. Consider an *N*×*N* grid of sites, with site vacancy probablity *p*. The system is said to percolate **iff** the top and bottom are connected by open sites.

**Percolation Phase Transition**

When *N* is large, theory guarantees a sharp threshold *p*\* such that for
* *p* > *p*\*: almost certainly percolates
* *p* < *p*\*: almost certainly doesn't percolate.

We determine *p*\* by employing the union-find algorithm in Monte-Carlo simulation.
1. Initialize *N*×*N* whole grid to be blocked.
  1. Create an object for each site, indexing them from 0 to *N*^2-1.
2. Randomly open sites until the top and bottom are connected.
  1. Sites are in the same component if connected by open sites.
  2. ~~Percolates **iff** any site in the bottom row is connected to a site in the top row. This brute force solution ends up being quadratic in time.~~ The trick is to introduce two virtual sites, one connected to every site in the top row and the other connected to every site in the bottom row. The system percolates **iff** the virtual top site is connected to the virtual bottom site.
  3. Model opening a new site as connecting to all adjacent open sites.
3. The vacancy percentage estimates *p*\*.

The computational solution for *p*\* for large square lattices is approximately 0.592746.


## Week 1: Analysis of Algorithms